### Tarea 2, Métodos Numéricos, Miguel Angel Ruiz Ortiz

### Problema 2.a)

-> Implementa el método de eliminación Gaussiana junto con un método de sustitución para resolver sistemas lineales. Estima para un sistema de n-ecuaciones cuántas operaciones requiere.

La estimación de operaciones se encuentra en el PDF.

In [21]:
import numpy as np

In [22]:
def solveUpperTriangular(A: np.array, b: np.array, TOL: float = 10**(-5)) -> np.array:
    """Resuelve Ax = b cuando A es una matriz triangular superior de nxn, y b es un vector de tamaño n, mediante sustitución regresiva.
    Regresa la solución x si existe, y en otro caso levanta una excepción.

    :param A: Matriz triangular superior de nxn
    :type A: np.array
    :param b: Vector de tamaño n
    :type b: np.array
    :param TOL: Toleracia, i.e., si |x|<TOL, entonces x se considera como 0
    :type TOL: float
    :return: Vector solución x de tamaño n (si existe). 
    :rtype: np.array
    """
    
    n: int = b.size
    x: np.array = np.zeros(n)
    
    for i in reversed(range(n)):
        if abs(A[i, i]) < TOL: # checar si A[i,i] es prácticamente 0
            raise Exception("No existe una solución única al sistema")
        
        s = A[i,i+1:]@x[i+1:] # \sum_{j=i+1}^{n-1} a_{ij}x_j corresponde al producto punto entre A[i,i+1:] y x[i+1:]
        # notar que cuando i == n-1, el producto punto (de arrays vacíos) da 0

        x[i] = (b[i] - s)/A[i,i]
        
    return x

In [23]:
def solveByGaussianElimination(A: np.array, b: np.array, TOL: float = 10**(-5)) -> np.array:
    """Resuelve Ax=b mediante eliminación Gaussiana con pivoteo parcial y sustitución regresiva. Regresa la solución x si existe, y en otro caso levanta una excepción.

    :param A: Matriz de nxn
    :type A: np.array
    :param b: Vector independiente de tamaño n
    :type b: np.array
    :param TOL: Toleracia, i.e., si |x|<TOL, entonces x se considera como 0
    :type TOL: float
    :return: Vector solución x de tamaño n (si existe)
    :rtype: np.array
    """
    # copiamos A, b y casteamos a flotante, para no modificar la matriz y vector originales mientras hacemos eliminación Gaussiana, 
    # y también para no hacer división entera si A tiene puros enteros (dtype 'int64')
    A_c = A.astype('float64')
    b_c = b.astype('float64') 
    
    n: int = b.size
    
    for i in range(n):
        i_max = np.abs(A_c[i:, i]).argmax() + i # encontramos la fila del elemento de mayor magnitud en la columna i después de o en la fila i
        if i_max != i:
            A_c[[i, i_max]] = A_c[[i_max, i]] # swap de fila i y fila i_max
            b_c[i], b_c[i_max] = b_c[i_max], b_c[i]
        
        if abs(A_c[i, i]) < TOL: # checar si A_c[i,i] es prácticamente 0
            raise Exception("No existe una solución única al sistema") # no existe porque si el elemento de mayor magnitud es cercano a 0, los demás también
        
        for k in range(i+1, n):
            # hacer 0's debajo del pivote A_c[i,i]
            m_ki = A_c[k, i]/A_c[i,i]
            A_c[k,i:] = A_c[k,i:] - m_ki*A_c[i,i:]
            b_c[k] = b_c[k] - m_ki*b_c[i]

    # aqui ya tendriamos un sistema Ux = c, donde U es triangular superior
    return solveUpperTriangular(A_c, b_c, TOL)
        

Testing con sistema de ecuaciones del Problema 1

In [24]:

p, q = 2, 2
A = np.array([[1, 1, 2, 0], [0, 1, 1, 2], [1, 1, 3, 3], [0, 2, 5, p]])
b = np.array([q, 0, 0, 3])
U = np.array([[1, 1, 2, 0], [0, 1, 1, 2], [0, 0, 1, 3], [0, 0, 0, p-13]]) # U es la matriz diagonal que surge de eliminación Gaussiana sin utilizar pivoteo parcial
c = np.array([q, 0, -q, 3+3*q]) # c es el vector independiente que surge de eliminación Gaussiana sin utilizar pivoteo parcial
print("A:\n", A)
print("U:\n", U)

A:
 [[1 1 2 0]
 [0 1 1 2]
 [1 1 3 3]
 [0 2 5 2]]
U:
 [[  1   1   2   0]
 [  0   1   1   2]
 [  0   0   1   3]
 [  0   0   0 -11]]


In [25]:
# Expresion analítica de la solución:
np.array([
    2*q+15*(1+q)/(p-13), 
    q + 3*(1+q)/(p-13),
    -q-9*(1+q)/(p-13),
    3*(1+q)/(p-13)
])

array([-0.09090909,  1.18181818,  0.45454545, -0.81818182])

In [26]:
# Resolviendo Ux = c
solveUpperTriangular(U, c)

array([-0.09090909,  1.18181818,  0.45454545, -0.81818182])

In [27]:
# Resolviendo con eliminación Gaussiana y pivoteo parcial:
solveByGaussianElimination(A, b)

array([-0.09090909,  1.18181818,  0.45454545, -0.81818182])

### Problema 3

-> 3.a) Encontrar $A^{-1}$ y resolver $Ax = b$

In [28]:
A = np.array([[4, -1, 0, 0], [-1, 4, -1, 0], [0, -1, 4, -1], [0, 0, -1, 3]], dtype="object")
b = np.array([15, 10, 10, 10], dtype="object")

In [29]:
from fractions import Fraction # modulo de Python para trabajar con racionales

In [30]:
A_inv = np.array([
    [Fraction(41, 153), Fraction(11, 153), Fraction(1, 51), Fraction(1, 153)], 
    [Fraction(11, 153), Fraction(44, 153), Fraction(4, 51), Fraction(4, 153)], 
    [Fraction(1, 51), Fraction(4, 51), Fraction(5, 17), Fraction(5, 51)], 
    [Fraction(1, 153), Fraction(4, 153), Fraction(15, 153), Fraction(56, 153)]
    ], dtype="object") # inversa de A
A_inv

array([[Fraction(41, 153), Fraction(11, 153), Fraction(1, 51),
        Fraction(1, 153)],
       [Fraction(11, 153), Fraction(44, 153), Fraction(4, 51),
        Fraction(4, 153)],
       [Fraction(1, 51), Fraction(4, 51), Fraction(5, 17),
        Fraction(5, 51)],
       [Fraction(1, 153), Fraction(4, 153), Fraction(5, 51),
        Fraction(56, 153)]], dtype=object)

In [31]:
# verificar que A_inv es la inversa de A
print(A@A_inv)
print(A_inv@A)

[[Fraction(1, 1) Fraction(0, 1) Fraction(0, 1) Fraction(0, 1)]
 [Fraction(0, 1) Fraction(1, 1) Fraction(0, 1) Fraction(0, 1)]
 [Fraction(0, 1) Fraction(0, 1) Fraction(1, 1) Fraction(0, 1)]
 [Fraction(0, 1) Fraction(0, 1) Fraction(0, 1) Fraction(1, 1)]]
[[Fraction(1, 1) Fraction(0, 1) Fraction(0, 1) Fraction(0, 1)]
 [Fraction(0, 1) Fraction(1, 1) Fraction(0, 1) Fraction(0, 1)]
 [Fraction(0, 1) Fraction(0, 1) Fraction(1, 1) Fraction(0, 1)]
 [Fraction(0, 1) Fraction(0, 1) Fraction(0, 1) Fraction(1, 1)]]


In [32]:
# resolver Ax = b utilizando la inversa de A
x = A_inv@b
x

array([Fraction(5, 1), Fraction(5, 1), Fraction(5, 1), Fraction(5, 1)],
      dtype=object)

In [33]:
A@x

array([Fraction(15, 1), Fraction(10, 1), Fraction(10, 1), Fraction(10, 1)],
      dtype=object)

-> 3.b) Número de condición de la matriz A, considerando la norma infinito

In [34]:
norm_inf_A = np.linalg.norm(A, np.inf)
norm_inf_A

6

In [35]:
norm_inf_A_inv = np.linalg.norm(A_inv, np.inf)
norm_inf_A_inv

Fraction(76, 153)

In [36]:
norm_inf_A*norm_inf_A_inv # número de condición de A

Fraction(152, 51)

In [37]:
float(norm_inf_A*norm_inf_A_inv) # número de condición de A

2.980392156862745

-> 3.c) Factorización de Cholesky

In [38]:
L = np.array([
    [1, 0, 0, 0],
    [-1/4, 1, 0, 0],
    [0, -4/15, 1, 0],
    [0, 0, -15/56, 1]
    ], dtype='float64')
D_sq = np.diag([2, (15/4)**0.5, (56/15)**0.5, (153/56)**0.5])
L_chol = L@D_sq
L_chol #matriz L de factorización de Cholesky

array([[ 2.        ,  0.        ,  0.        ,  0.        ],
       [-0.5       ,  1.93649167,  0.        ,  0.        ],
       [ 0.        , -0.51639778,  1.93218357,  0.        ],
       [ 0.        ,  0.        , -0.51754917,  1.6529195 ]])

In [39]:
L_chol@L_chol.T # comprobación de que A = L*L^T

array([[ 4., -1.,  0.,  0.],
       [-1.,  4., -1.,  0.],
       [ 0., -1.,  4., -1.],
       [ 0.,  0., -1.,  3.]])

In [40]:
np.array([
    [2, 0, 0, 0],
    [-1/2, 15**0.5/2, 0, 0],
    [0, -2/(15**0.5), (2*14**0.5)/(15**0.5), 0],
    [0, 0, -15**0.5/(2*14**0.5), 3*17**(0.5)/(2*14**0.5)]
    ], dtype='float64') # matriz L de factorización de Cholesky (explícita)

array([[ 2.        ,  0.        ,  0.        ,  0.        ],
       [-0.5       ,  1.93649167,  0.        ,  0.        ],
       [ 0.        , -0.51639778,  1.93218357,  0.        ],
       [ 0.        ,  0.        , -0.51754917,  1.6529195 ]])